available here in pdf: 


https://arkhamarchivist.com/ebook/The%20Complete%20Works%20of%20H.P.%20Lovecraft.pdf

working with pdf: 

https://realpython.com/pdf-python/

had to pip install pypdf2, pip install pdfminer.six first

In [48]:
import PyPDF2
import data_func
import csv

In [2]:
reader = PyPDF2.PdfFileReader('./data/original/Complete_Works_Lovecraft.pdf')

In [3]:
reader.documentInfo

{'/Author': 'H.P. Lovecraft',
 '/Creator': 'Microsoft® Word 2010',
 '/CreationDate': "D:20110729214233-04'00'",
 '/ModDate': "D:20110729214233-04'00'",
 '/Producer': 'Microsoft® Word 2010'}

In [4]:
num_of_pages = reader.numPages
num_of_pages

708

In [ ]:
# to extract with pypdf2, we can use: reader2.getPage(7).extractText(), but lines will be missing

In [5]:
# first, we need to extract the page numbers from the table of contents

In [5]:
# saving a new table of contents file
writer = PyPDF2.PdfFileWriter()

for page in range(2,4):

    writer.addPage(reader.getPage(page))
    
output_filename = './data/original/table_of_contents.pdf'

with open(output_filename, 'wb') as output:
    writer.write(output)

In [ ]:
# read in table_of_contents file with pdfminer

In [6]:
text = data_func.convert_pdf_to_txt('./data/original/table_of_contents.pdf')

In [7]:
text = text.replace('.','')
text = text.replace('\x0c','')

In [8]:
table_of_contents_raw = text.split('\n')

In [9]:
table_of_contents_raw

['Table of Contents ',
 '',
 'Preface  2 ',
 'The Tomb  5 ',
 'Dagon  12 ',
 'Polaris 16 ',
 'Beyond the Wall of Sleep  19 ',
 'Memory  26 ',
 'Old Bugs  27 ',
 'The Transition of Juan Romero  32 ',
 'The White Ship  37 ',
 'The Doom That Came to Sarnath  41 ',
 'The Statement of Randolph Carter  45 ',
 'The Terrible Old Man  49 ',
 'The Tree  51 ',
 'The Cats of Ulthar 54 ',
 'The Temple  56 ',
 'Facts Concerning the Late Arthur Jermyn and His Family  64 ',
 'The Street  70 ',
 'Celephaïs  74 ',
 'From Beyond  78 ',
 'Nyarlathotep  83 ',
 'The Picture in the House  85 ',
 'Ex Oblivione  90 ',
 'The Nameless City  92 ',
 'The Quest of Iranon  100 ',
 'The Moon-Bog  104 ',
 'The Outsider  109 ',
 'The Other Gods  113 ',
 'The Music of Erich Zann  116 ',
 'Herbert West — Reanimator  121 ',
 'Hypnos  139 ',
 'What the Moon Brings  144 ',
 'Azathoth  146 ',
 'The Hound  147 ',
 'The Lurking Fear  152 ',
 'The Rats in the Walls  165 ',
 'The Unnamable  177 ',
 'The Festival  182 ',
 'The Sh

In [ ]:
# divide all of them with '  ', see if there is a 2nd element, if yes, if second is a number, and if yes, split them

In [ ]:
# unfortunately not all elements are only 2 space

In [55]:
title_list = []
pagenum_list = []
title_formatted_list = []
for item in table_of_contents_raw:
        title, pagenum = data_func.split_to_title_and_pagenum(item)
        if title != None:
            title_list.append(title)
            pagenum_list.append(pagenum)
            title_formatted_list.append(data_func.convert_title_to_filename(title))
            
# for page_list, we need to add the last page as well
pagenum_list.append(num_of_pages + 1)

In [56]:
# let's save this all in a csv too

In [57]:
with open('./data/original/table_of_contents.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerows(zip(title_list, pagenum_list, title_formatted_list))

In [58]:
# saving the individual pdf files
# we are skipping the first one, which is preface
for i in range(1, len(title_formatted_list)):
    title_formatted = title_formatted_list[i]
    page_start = pagenum_list[i] - 1
    page_end = pagenum_list[i+1] - 2
    
    writer = PyPDF2.PdfFileWriter()

    for page in range(page_start,page_end + 1):
        writer.addPage(reader.getPage(page))
    
    output_filename = './data/original/pdfs/' + title_formatted + '.pdf'

    with open(output_filename, 'wb') as output:
        writer.write(output)
    

In [59]:
# now we cycle through them, and save everything in a txt

In [68]:
for title_formatted in title_formatted_list[1:]:
    
    text = data_func.convert_pdf_to_txt('./data/original/pdfs/' + title_formatted + '.pdf')
    
    # replace 'Return to Table of Contents', which is not part of the text
    text = text.replace('Return to Table of Contents', '')
    
    # replace Fin from the end of the last title
    if title_formatted == 'the_haunter_of_the_dark':
        text = text[:-15]
    
    text_file = open('./data/original/txts/' + title_formatted + '.txt', 'w')
    n = text_file.write(text)
    text_file.close()

In [69]:
# OK, txts are done